In [2]:
import json
import requests
import pandas as pd

### Get JSON respresentation of Drosophila anatomy ontology

Drosophila anatomy ontology, json representation is pulled from a persistent URL.  This will always pull the latest version.  To refer to this version in future, use the version IRI (below) with extension -> .json,



In [3]:
fbbt_file = requests.get("http://purl.obolibrary.org/obo/fbbt/fbbt.json") # Persistent URL
fbbt_graphs = json.loads(fbbt_file.text)
fbbt = fbbt_graphs['graphs'][0]
fbbt['meta']['version']

'http://purl.obolibrary.org/obo/fbbt/releases/2018-02-21/fbbt.owl'

### Generate pandas DataFrame with pre and post columns & save as csv

* Class level representation needs to be simplified to ignore quantifiers:
   * All X synapsed_to some Y; all A synapsed_by some B -> pre:X post:Y , pre:B - post:A
* This version uses full iri's in each column.  For labels use node['lbl']


In [6]:
lookup = { n['lbl']: n['id'] for n in fbbt['nodes'] if 'lbl' in n.keys()}
synapsed_by = [ {'pre': e['obj'], 'post': e['sub']} for e in fbbt['edges'] if lookup['synapsed_by'] == e['pred'] ]
synapsed_to = [ {'pre': e['sub'], 'post': e['obj']} for e in fbbt['edges'] if lookup['synapsed_to'] == e['pred'] ]
sb = pd.DataFrame.from_records(synapsed_by)
all = sb.append(pd.DataFrame.from_records(synapsed_to))
all.drop_duplicates(inplace=True)

all.to_csv("connection_tab.tsv", sep="\t", index=False)

URLs resolve to OntoBee, which can be used to programatically retrieive XML for the term. * Some other recipes for URLS:
    * VFB: http://purl.obolibrary.org/obo/FBbt_00003361  -> http://virtualflybrain.org/reports/FBbt_00003361 
    * Ontology Lookup Service (OLS):
      * WEB https://www.ebi.ac.uk/ols/ontologies/fbbt/terms?iri=http://virtualflybrain.org/reports/FBbt_00003361
      * API term request recipe (from iri) here: https://www.ebi.ac.uk/ols/docs/api#resources-term
It would be reasonably straightforward to get the same results using OLS API calls rather than working directly with the JSON representation.  We also have an open Cypher Endpoint for VFB that could be used to query this info, but I'll refrain from sharing for now as there will be some churn in the schema over the next few months. 
